In [42]:
import numpy as np

source = np.array([0.0, 0.0, 0.0])
direction = np.array([1/np.sqrt(14), 2/np.sqrt(14), 3/np.sqrt(14)])
grid = np.ones((101, 101, 101))
current_voxel = np.array([50, 50, 50])
voxel_size = np.array([1.0, 1.0, 1.0])
    

In [41]:
import numpy as np

def dda_3d(source, direction, grid, original_voxel, voxel_size):

    step = np.zeros(3)
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1

    current_voxel = np.copy(original_voxel)
    
    t = np.zeros(3)
    t[0] = (voxel_size[0] / 2) / direction[0]
    t[1] = (voxel_size[1] / 2) / direction[1]
    t[2] = (voxel_size[2] / 2) / direction[2]

    delta_t = np.zeros(3)
    delta_t[0] = voxel_size[0] / direction[0]
    delta_t[1] = voxel_size[1] / direction[1]
    delta_t[2] = voxel_size[2] / direction[2]

    xmax, ymax, zmax = grid.shape

    voxels_traversed = []
    intersection_t_values = []

    while (current_voxel[0] >= 0 and current_voxel[0] < xmax and
           current_voxel[1] >= 0 and current_voxel[1] < ymax and
           current_voxel[2] >= 0 and current_voxel[2] < zmax):

        voxels_traversed.append(np.copy(current_voxel))
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values.append(t[0])
                t[0] += delta_t[0]
                current_voxel[0] += step[0]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                current_voxel[2] += step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values.append(t[1])
                t[1] += delta_t[1]
                current_voxel[1] += step[1]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                current_voxel[2] += step[2]

    return intersection_t_values

In [46]:
%timeit dda_3d(source, direction, grid, current_voxel, voxel_size)

761 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
dda_3d(source, direction, grid, current_voxel, voxel_size)

[0.6236095644623235,
 0.9354143466934853,
 1.8708286933869704,
 1.8708286933869707,
 2.806243040080456,
 3.1180478223116177,
 4.365266951236265,
 4.677071733467427,
 5.612486080160912,
 5.612486080160912,
 6.547900426854397,
 6.8597052090855595,
 8.106924338010206,
 8.418729120241368,
 9.354143466934852,
 9.354143466934854,
 10.289557813628338,
 10.601362595859499,
 11.848581724784145,
 12.160386507015309,
 13.095800853708791,
 13.095800853708795,
 14.03121520040228,
 14.343019982633438,
 15.590239111558084,
 15.90204389378925,
 16.837458240482732,
 16.837458240482736,
 17.77287258717622,
 18.08467736940738,
 19.33189649833203,
 19.643701280563192,
 20.579115627256677,
 20.579115627256677,
 21.514529973950165,
 21.826334756181325,
 23.073553885105973,
 23.385358667337137,
 24.320773014030618,
 24.32077301403062,
 25.25618736072411,
 25.56799214295527,
 26.815211271879917,
 27.12701605411108,
 28.06243040080456,
 28.062430400804566,
 28.997844747498053,
 29.309649529729214,
 30.55686865

In [44]:
load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [49]:
import numpy as np

source = np.array([0.0, 0.0, 0.0], dtype=np.float64)
direction = np.array([1/np.sqrt(14), 2/np.sqrt(14), 3/np.sqrt(14)], dtype=np.float64)
grid = np.ones((101, 101, 101), dtype=np.float64)
current_voxel = np.array([50, 50, 50], dtype=np.int32)
voxel_size = np.array([1.0, 1.0, 1.0], dtype=np.float64)


In [21]:
%%cython
import numpy as np
cimport cython
cimport numpy as cnp

def dda_3d_naive(cnp.float64_t[:] source, cnp.float64_t[:] direction, cnp.float64_t[:,:,:] grid, cnp.int32_t[:] current_voxel, cnp.float64_t[:] voxel_size):

    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1
    
    cdef cnp.int32_t vox[3]
    vox[0] = current_voxel[0]
    vox[1] = current_voxel[1]
    vox[2] = current_voxel[2]

    cdef cnp.float64_t t[3]
    t[0] = (voxel_size[0] / 2) / direction[0]
    t[1] = (voxel_size[1] / 2) / direction[1]
    t[2] = (voxel_size[2] / 2) / direction[2]

    cdef cnp.float64_t delta_t[3]
    delta_t[0] = voxel_size[0] / direction[0]
    delta_t[1] = voxel_size[1] / direction[1]
    delta_t[2] = voxel_size[2] / direction[2]

    cdef cnp.float64_t xmax = grid.shape[0]
    cdef cnp.float64_t ymax = grid.shape[1]
    cdef cnp.float64_t zmax = grid.shape[2]

    cdef list voxels_traversed = []
    cdef list intersection_t_values = []
    
    while (vox[0] >= 0 and vox[0] < xmax and
           vox[1] >= 0 and vox[1] < ymax and
           vox[2] >= 0 and vox[2] < zmax):
        
        voxels_traversed.append(np.copy(vox))
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values.append(t[0])
                t[0] += delta_t[0]
                vox[0] = vox[0] + step[0]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values.append(t[1])
                t[1] += delta_t[1]
                vox[1] = vox[1] + step[1]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]

    return intersection_t_values


In [22]:
%timeit dda_3d_naive(source, direction, grid, current_voxel, voxel_size)

228 µs ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
%%cython

import numpy as np
cimport cython
cimport numpy as cnp
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)

def dda_3d_fast(cnp.float64_t[:] source, cnp.float64_t[:] direction, cnp.float64_t[:,:,:] grid, cnp.int32_t[:] current_voxel, cnp.float64_t[:] voxel_size):

    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1
    
    cdef cnp.int32_t vox[3]
    vox[0] = current_voxel[0]
    vox[1] = current_voxel[1]
    vox[2] = current_voxel[2]

    cdef cnp.float64_t t[3]
    t[0] = (voxel_size[0] / 2) / direction[0]
    t[1] = (voxel_size[1] / 2) / direction[1]
    t[2] = (voxel_size[2] / 2) / direction[2]

    cdef cnp.float64_t delta_t[3]
    delta_t[0] = voxel_size[0] / direction[0]
    delta_t[1] = voxel_size[1] / direction[1]
    delta_t[2] = voxel_size[2] / direction[2]

    cdef cnp.float64_t xmax = grid.shape[0]
    cdef cnp.float64_t ymax = grid.shape[1]
    cdef cnp.float64_t zmax = grid.shape[2]

    cdef cnp.int32_t voxels_traversed[200][3]
    cdef cnp.float64_t intersection_t_values[200]
    
    cdef cnp.int32_t v_count = 0 
    cdef cnp.int32_t i_count = 0
    
    while (vox[0] >= 0 and vox[0] < xmax and
           vox[1] >= 0 and vox[1] < ymax and
           vox[2] >= 0 and vox[2] < zmax):

        voxels_traversed[v_count][0] = vox[0]
        voxels_traversed[v_count][1] = vox[1]
        voxels_traversed[v_count][2] = vox[2]
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values[i_count] = t[0]
                t[0] += delta_t[0]
                vox[0] = vox[0] + step[0]
            else:
                intersection_t_values[i_count] = t[2]
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values[i_count] = t[1]
                t[1] += delta_t[1]
                vox[1] = vox[1] + step[1]
            else:
                intersection_t_values[i_count] = t[2]
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        v_count = v_count + 1
        i_count = i_count + 1
    
    return intersection_t_values

In [50]:
%timeit dda_3d_fast(source, direction, grid, current_voxel, voxel_size)

4.44 µs ± 44.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
dda_3d_fast(source, direction, grid, current_voxel, voxel_size)

[0.6236095644623235,
 0.9354143466934853,
 1.8708286933869704,
 1.8708286933869707,
 2.806243040080456,
 3.1180478223116177,
 4.365266951236265,
 4.677071733467427,
 5.612486080160912,
 5.612486080160912,
 6.547900426854397,
 6.8597052090855595,
 8.106924338010206,
 8.418729120241368,
 9.354143466934852,
 9.354143466934854,
 10.289557813628338,
 10.601362595859499,
 11.848581724784145,
 12.160386507015309,
 13.095800853708791,
 13.095800853708795,
 14.03121520040228,
 14.343019982633438,
 15.590239111558084,
 15.90204389378925,
 16.837458240482732,
 16.837458240482736,
 17.77287258717622,
 18.08467736940738,
 19.33189649833203,
 19.643701280563192,
 20.579115627256677,
 20.579115627256677,
 21.514529973950165,
 21.826334756181325,
 23.073553885105973,
 23.385358667337137,
 24.320773014030618,
 24.32077301403062,
 25.25618736072411,
 25.56799214295527,
 26.815211271879917,
 27.12701605411108,
 28.06243040080456,
 28.062430400804566,
 28.997844747498053,
 29.309649529729214,
 30.55686865